In [1]:
from azure.ai.ml import MLClient, command, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import (
    AmlCompute,
    IdentityConfiguration,
    Environment,
    Model,
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration
)
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

In [2]:
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="814b44e2-bd64-4baa-b7eb-954e6b82a85e",
    resource_group_name="mlops-demo-rg",
    workspace_name="mlw-dev"
)

## Infrastructure - Compute

In [ ]:
# configure
compute_clu = AmlCompute(
    name="clu-small-cpu",
    type="amlcompute",
    size="Standard_D2_v3",
    location="centralus", # should be same as workspace
    tier="Dedicated",
    identity=IdentityConfiguration(type="system_assigned"),
    min_instances=0,
    max_instances=2,
    idle_time_before_scale_down=120,
    ssh_public_access_enabled=False,
)

# create
ml_client.begin_create_or_update(compute_clu).result()

## Infrastructure - Environment

In [ ]:
# configure
env_docker_conda = Environment(
    name="mlops-demo-train",
    version="3",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="../environments/requirements-train.yaml",
)

# create
ml_client.environments.create_or_update(env_docker_conda)

## Components

In [49]:
preprocess_component = command(
    name="process_data",
    display_name="Data Processor",
    inputs={
        "train_size": Input(type="number")
    },
    outputs={
        "train_data": Output(type="uri_folder"),
        "test_data": Output(type="uri_folder"),
    },
    code="../src",
    command="""python preprocess.py \
    --train_size ${{inputs.train_size}} \
    --train_data ${{outputs.train_data}} \
    --test_data ${{outputs.test_data}} \
    """,
    compute="clu-small-cpu",
    environment="mlops-demo-train:3",
)

In [50]:
train_component = command(
    name="train_model",
    display_name="Model Trainer",
    inputs={
        "input_data": Input(type="uri_folder")
    },
    outputs={
        "output_model": Output(type="uri_folder")
    },
    code="../src",
    command="""python train.py \
    --input_data ${{inputs.input_data}} \
    --output_model ${{outputs.output_model}} \
    """,
    compute="clu-small-cpu",
    environment="mlops-demo-train:3"
)

In [51]:
evaluate_component = command(
    name="eval_model",
    display_name="Model Evaluator",
    inputs={
        "input_model": Input(type="uri_folder"),
        "input_data": Input(type="uri_folder")
    },
    code="../src",
    command="""python evaluate.py \
    --input_model ${{inputs.input_model}} \
    --input_data ${{inputs.input_data}} \
    """,
    compute="clu-small-cpu",
    environment="mlops-demo-train:3"
)

## Pipeline

In [52]:
@pipeline
def train_pipeline(train_size: float):

    preprocess_job = preprocess_component(
        train_size=train_size
    )

    train_job = train_component(
        input_data=preprocess_job.outputs.train_data
    )

    evaluate_job = evaluate_component(
        input_data=preprocess_job.outputs.test_data,
        input_model=train_job.outputs.output_model
    )

    return {"model": train_job.outputs.output_model}

In [ ]:
pipeline_job = train_pipeline(
    train_size=0.80
)

pipeline_job = ml_client.jobs.create_or_update(pipeline_job, experiment_name="aml-pipelines")

## Register Model

In [ ]:
pipeline_job.name

In [48]:
model_name = "bow-sentiment"

In [ ]:
pipeline_model = Model(
    path=f"azureml://jobs/{pipeline_job.name}/outputs/model/model_pipeline.pkl", # job id from pipeline
    name=model_name,
    version="1"
)

ml_client.models.create_or_update(pipeline_model)

In [49]:
registered_model = ml_client.models.get(name=model_name, version=1)

In [ ]:
registered_model.path

## Create Endpoint

In [5]:
endpoint_name = "sentiment-ep"

In [ ]:
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## Create Inf Env

In [ ]:
# configure
env_inf = Environment(
    name="mlops-demo-inf",
    version="1",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="../environments/requirements-inf.yaml",
)

# create
ml_client.environments.create_or_update(env_inf)

## Create Deployment

In [87]:
deployment_name = "red"

In [88]:
deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=f"{model_name}:1",
    environment="mlops-demo-inf:1",
    code_configuration=CodeConfiguration(
        code="../src", scoring_script="score.py"
    ),
    instance_type="Standard_E2s_v3",
    instance_count=1,
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

In [77]:
logs = ml_client.online_deployments.get_logs(
    name=deployment_name,
    endpoint_name=endpoint_name,
    lines=50
)

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {deployment_name: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)